In [ ]:
#fastwhisper and gemini
!pip install -q faster-whisper google-generativeai
!sudo apt update -y && sudo apt install -y ffmpeg


from faster_whisper import WhisperModel
import os
import json
import google.generativeai as genai
from google.colab import drive
from pathlib import Path

# Load environment variables from .env
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Check if the key is loaded
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found. Make sure it is in your .env file.")

#Loading FastWhisper
whisper_model = WhisperModel("medium", device="cpu", compute_type="int8")

# Configure Gemini API
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

#transcribing(Fastwhisper)
def transcribe_hindi_fastwhisper(audio_path):
    segments, _ = whisper_model.transcribe(audio_path, language="hi")
    return " ".join(segment.text for segment in segments)

#Gemini(Correction,Translation)
def correct_and_translate(text_hindi):
    prompt = f"""
    You're a Hindi-English translator. The following text is a Hindi speech transcription.
    Step 1: Correct spelling/grammar/punctuation errors in Hindi.
    Step 2: Translate the corrected Hindi to professional English.

    Hindi Input:
    {text_hindi}

    Respond as:
    Hindi (Corrected):
    <corrected_hindi>

    English Translation:
    <translated_english>
    """
    response = gemini_model.generate_content(prompt)
    return response.text.strip()

#Audio
base_path = Path("/content/drive/MyDrive/C-DAC")
audio_files = [base_path / name for name in [
    "Root_1_6130579.mp3", "6131490.mp3", "6131551.mp3", "6131785.mp3", "6131832.mp3", "6131851.mp3",
    "6131888.mp3", "6132149.mp3", "6132376.mp3", "6132569.mp3", "6132577.mp3", "6132622.mp3",
    "6132729.mp3", "6132735.mp3", "6132745.mp3", "6132751.mp3", "6132811.mp3", "6133246.mp3",
    "6133493.mp3", "6133498.mp3", "6133522.mp3", "6133557.mp3", "6133644.mp3", "6133757.mp3",
    "6133863.mp3", "6134064.mp3"
]]


resume_path = "/content/drive/MyDrive/C-DAC/asr_translations_partial.json"

if os.path.exists(resume_path):
    with open(resume_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_files = {item["file"] for item in results}
    print(f" Skipping {len(done_files)} already processed files.")
else:
    results = []
    done_files = set()
    print(" Starting fresh...")

audio_files = [f for f in audio_files if f.name not in done_files]


results = []
for file in audio_files:
    try:
        print(f" Processing: {file}")
        hindi_raw = transcribe_hindi_fastwhisper(str(file))
        translated_output = correct_and_translate(hindi_raw)
        results.append({
            "file": file.name,
            "hindi_raw": hindi_raw,
            "translation": translated_output
        })
    except Exception as e:
        print(f" Error processing {file.name}: {e}")

#  Saving JSON to Drive
output_path = "/content/drive/MyDrive/C-DAC/asr_translations_fast.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Fast ASR + Translation complete. File saved to: {output_path}")


JSON TO .TXT

In [ ]:
import json

input_path = "/content/drive/MyDrive/C-DAC/asr_translations_fast.json"
output_path = "/content/drive/MyDrive/C-DAC/english_translations.txt"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)


translations = []
for item in data:
    full = item["translation"]
    # Split and keep only the final English part after "**English Translation:**"
    if "**English Translation:**" in full:
        english_part = full.split("**English Translation:**")[-1].strip()
    else:
        english_part = full.strip()
    translations.append(english_part)

#saving to .TXT
with open(output_path, "w", encoding="utf-8") as f:
    for line in translations:
        f.write(line + "\n\n")

print(f" Extracted {len(translations)} English translations to: {output_path}")


CLEAN, CHUNK AND STORE THE TEXT

In [ ]:
from pathlib import Path

#Paths
input_path = "/content/drive/MyDrive/C-DAC/ASR Translations Fast.txt"
root_output_path = "/content/drive/MyDrive/C-DAC/root_chunks.txt"
comment_output_path = "/content/drive/MyDrive/C-DAC/comment_chunks.txt"

#Read and split input
with open(input_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

root_text = lines[0]
comment_texts = lines[1:]

#  Chunking helper
def chunk_text(text, chunk_size=200, overlap=40):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

#clean & chunk Root
root_chunks = chunk_text(root_text, chunk_size=200, overlap=40)

#clean & chunk Comments
comment_chunks = []
for comment in comment_texts:
    comment_chunks.extend(chunk_text(comment, chunk_size=200, overlap=40))

#Save root chunks
with open(root_output_path, "w", encoding="utf-8") as f:
    for chunk in root_chunks:
        f.write(chunk + "\n")

#Save comment chunks
with open(comment_output_path, "w", encoding="utf-8") as f:
    for chunk in comment_chunks:
        f.write(chunk + "\n")

print(f"Root chunks saved: {root_output_path} ({len(root_chunks)} chunks)")
print(f"Comment chunks saved: {comment_output_path} ({len(comment_chunks)} chunks)")


RAG - Pipeline => create vector embeddings -> store it (ALL-MPNet-base for embeddings and FAISS or chroma for storing)

In [ ]:
!pip install -q sentence-transformers faiss-cpu


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

#Load root and comment chunks
def load_chunks(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

root_chunks = load_chunks("/content/drive/MyDrive/C-DAC/root_chunks.txt")
comment_chunks = load_chunks("/content/drive/MyDrive/C-DAC/comment_chunks.txt")

#Load MPNet model
model = SentenceTransformer("all-mpnet-base-v2")


print("Embedding root chunks...")
root_embeddings = model.encode(root_chunks, show_progress_bar=True, convert_to_numpy=True)

print("Embedding comment chunks...")
comment_embeddings = model.encode(comment_chunks, show_progress_bar=True, convert_to_numpy=True)

# Create FAISS index
dimension = root_embeddings.shape[1]
root_index = faiss.IndexFlatL2(dimension)
comment_index = faiss.IndexFlatL2(dimension)

root_index.add(root_embeddings)
comment_index.add(comment_embeddings)


faiss.write_index(root_index, "/content/drive/MyDrive/C-DAC/faiss_root.index")
faiss.write_index(comment_index, "/content/drive/MyDrive/C-DAC/faiss_comment.index")

print("FAISS indices created and saved.")


Cosine similarity for comment embeddings and root embeddings - to reject or accept comment

In [ ]:
import faiss
import numpy as np
import json


SIMILARITY_THRESHOLD = 0.61

#FAISS indexes
root_index = faiss.read_index("/content/drive/MyDrive/C-DAC/faiss_root.index")
comment_index = faiss.read_index("/content/drive/MyDrive/C-DAC/faiss_comment.index")

#Load comment text chunks
with open("/content/drive/MyDrive/C-DAC/comment_chunks.txt", "r", encoding="utf-8") as f:
    comment_chunks = [line.strip() for line in f if line.strip()]

#comment embeddings
comment_embeddings = [comment_index.reconstruct(i) for i in range(comment_index.ntotal)]
comment_embeddings = np.stack(comment_embeddings)

#root embeddings
root_embeddings = [root_index.reconstruct(i) for i in range(root_index.ntotal)]
root_embeddings = np.stack(root_embeddings)

#Cosine similarity
def cosine_similarity_matrix(A, B):
    A_norm = A / np.linalg.norm(A, axis=1, keepdims=True)
    B_norm = B / np.linalg.norm(B, axis=1, keepdims=True)
    return np.dot(A_norm, B_norm.T)

similarities = cosine_similarity_matrix(comment_embeddings, root_embeddings)


results = []
for i, row in enumerate(similarities):
    max_sim = np.max(row)
    decision = "ACCEPT" if max_sim >= SIMILARITY_THRESHOLD else "REJECT"
    results.append({
        "comment": comment_chunks[i],
        "max_similarity": float(max_sim),
        "decision": decision
    })

#outputs
json_path = "/content/drive/MyDrive/C-DAC/comment_relevance_results.json"
txt_path = "/content/drive/MyDrive/C-DAC/comment_accepted.txt"

#JSON results saved
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

#accepted comments saved to TXT
with open(txt_path, "w", encoding="utf-8") as f:
    for r in results:
        if r["decision"] == "ACCEPT":
            f.write(r["comment"] + "\n")

print(" Relevance filtering complete.")
print(f" Saved detailed results to: {json_path}")
print(f" Saved accepted comments to: {txt_path}")


NOVELTY CHECK

In [ ]:
import os
import json
from dotenv import load_dotenv
import google.generativeai as genai

# Load API key from .env
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found. Please set it in your .env file.")

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

# Load accepted comments
with open("/content/drive/MyDrive/C-DAC/comment_accepted.txt", "r", encoding="utf-8") as f:
    accepted_comments = [line.strip() for line in f if line.strip()]

novel_comments = []
log = []

# Novelty detection loop
for idx, current in enumerate(accepted_comments):
    if idx == 0:
        novel_comments.append(current)
        log.append({
            "comment": current,
            "decision": "NOVEL",
            "reason": "First comment – assumed novel by default"
        })
        continue

    previous_context = "\n".join(novel_comments)

    prompt = f"""
You are a helpful research assistant. We are filtering out duplicate opinions from a discussion.

Below are earlier accepted unique comments:
{previous_context}

Now consider this new comment:
{current}

Question: Does this new comment add any novel point or perspective compared to the earlier ones?

Reply with either:
- "Duplicate" if it repeats existing ideas
- "Novel" if it adds a new point
"""

    try:
        response = model.generate_content(prompt)
        answer = response.text.strip().lower()

        if "novel" in answer:
            novel_comments.append(current)
            decision = "NOVEL"
        else:
            decision = "DUPLICATE"

        log.append({
            "comment": current,
            "decision": decision,
            "llm_response": response.text.strip()
        })

    except Exception as e:
        log.append({
            "comment": current,
            "decision": "ERROR",
            "error": str(e)
        })
        print(f"⚠️ Error with comment: {current}\n{e}")

# Save outputs
with open("/content/drive/MyDrive/C-DAC/comment_novel.txt", "w", encoding="utf-8") as f:
    for comment in novel_comments:
        f.write(comment + "\n")

with open("/content/drive/MyDrive/C-DAC/comment_novelty_log.json", "w", encoding="utf-8") as f:
    json.dump(log, f, indent=2, ensure_ascii=False)

print("Novelty check complete.")
print("Saved novel comments to: comment_novel.txt")
print("Full log saved to: comment_novelty_log.json")
